In [254]:
import numpy as np
import mlflow
import json
from mlflow.tracking import MlflowClient

# Initialisiere den Client
client = MlflowClient('/pfs/work7/workspace/scratch/ma_aansari-team_project_fss2024_de/mlflow')

In [270]:
common_corruptions = [
    'gaussian_noise',
  'shot_noise',
  'impulse_noise',
  'defocus_blur',
  'glass_blur',
  'motion_blur',
  'zoom_blur',
  'snow',
  'frost',
  'fog',
  'brightness',
  'contrast',
  'elastic_transform',
  'pixelate',
  'jpeg_compression'
]

severities = range(1, 2)

metrics = {'test_EPE_0':['cfnet', 'gwcnet-g'], 'test_Thres1_0':['cfnet', 'gwcnet-g'], 'test_Thres2_0':['cfnet', 'gwcnet-g'], 'test_Thres3_0':['cfnet', 'gwcnet-g'], 'test_D1_0':['cfnet', 'gwcnet-g']}

datasets = ['sceneflow']
architectures = ['cfnet', 'gwcnet-g', 'sttr', 'sttr-light']

# Assuming you have a way to fetch the mlflow data, for example, using mlflow.search_runs()
# Here, we will create a mock structure as an example

mlflow_data = {
    'architecture': [], 
    'backbone': [], 
    'type_backbone': [],
    'dataset': []
}

for metric in metrics.keys():
    for common_corruption in common_corruptions:
        for severity in severities:
            mlflow_data[f'{metric}_{common_corruption}_sev{severity}'] = []

print(mlflow_data)

{'architecture': [], 'backbone': [], 'type_backbone': [], 'dataset': [], 'test_EPE_0_gaussian_noise_sev1': [], 'test_EPE_0_shot_noise_sev1': [], 'test_EPE_0_impulse_noise_sev1': [], 'test_EPE_0_defocus_blur_sev1': [], 'test_EPE_0_glass_blur_sev1': [], 'test_EPE_0_motion_blur_sev1': [], 'test_EPE_0_zoom_blur_sev1': [], 'test_EPE_0_snow_sev1': [], 'test_EPE_0_frost_sev1': [], 'test_EPE_0_fog_sev1': [], 'test_EPE_0_brightness_sev1': [], 'test_EPE_0_contrast_sev1': [], 'test_EPE_0_elastic_transform_sev1': [], 'test_EPE_0_pixelate_sev1': [], 'test_EPE_0_jpeg_compression_sev1': [], 'test_Thres1_0_gaussian_noise_sev1': [], 'test_Thres1_0_shot_noise_sev1': [], 'test_Thres1_0_impulse_noise_sev1': [], 'test_Thres1_0_defocus_blur_sev1': [], 'test_Thres1_0_glass_blur_sev1': [], 'test_Thres1_0_motion_blur_sev1': [], 'test_Thres1_0_zoom_blur_sev1': [], 'test_Thres1_0_snow_sev1': [], 'test_Thres1_0_frost_sev1': [], 'test_Thres1_0_fog_sev1': [], 'test_Thres1_0_brightness_sev1': [], 'test_Thres1_0_cont

In [271]:
def get_experiment_id(architecture, dataset):
    experiment_name = f'Common_Corruptions_{architecture}_{dataset}'
    experiment_id = None

    experiment = client.get_experiment_by_name(experiment_name)
    if experiment:
        print(f"Experiment mit dem Namen {experiment_name} gefunden.")
        return experiment.experiment_id
    else:
        print(f"Experiment mit dem Namen {experiment_name} nicht gefunden.")
        return None

In [272]:
def get_run_id(architecture, dataset, corruption_type, severity, experiment_id):

    run_name = f"{architecture}_commoncorruption_{dataset}_{corruption_type}_{severity}"

    runs = client.search_runs(experiment_id, filter_string=f"tags.mlflow.runName = '{run_name}'", max_results=1)

    # Stelle sicher, dass der Run gefunden wurde
    if runs:
        print(f"Run mit dem Namen {run_name} gefunden.")
        return runs[0].info.run_id
    else:
        print(f"Run mit dem Namen {run_name} nicht gefunden.")
        return None

In [176]:
# Method to save time (only once loading experiment_id and run_id which is time consuming)
def get_mlflow_metadata():
    
    metadata = []

    for dataset in datasets:
        for architecture in architectures:
            experiment_id = get_experiment_id(architecture, dataset)
            
            for common_corruption in common_corruptions:
                for severity in severities:
                    run_id = get_run_id(architecture, dataset, common_corruption, severity, experiment_id=experiment_id)

                    metadata.append({
                        'experiment_id': experiment_id,
                        'run_id': run_id,
                        'dataset': dataset,
                        'architecture': architecture,
                        'common_corruption': common_corruption,
                        'severity': severity
                    })

    # Store mlflow_data as JSON
    with open('metadata.json', 'w') as f:
        json.dump(mlflow_data, f)
    
    return metadata

In [146]:
metadata = get_mlflow_metadata()
# metadata = json.load(open('metadata.json'))

Experiment mit dem Namen Common_Corruptions_cfnet_sceneflow gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_gaussian_noise_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_shot_noise_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_impulse_noise_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_defocus_blur_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_glass_blur_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_motion_blur_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_zoom_blur_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_snow_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_frost_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_fog_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_brightness_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_contrast_1 gefunden.
Run mit dem Namen cfnet_commoncorruption_sceneflow_el

In [273]:
def get_run_id_from_metadata(metadata, architecture, dataset, common_corruption, severity):
    
    for data in metadata:
        if data['dataset'] == dataset and data['architecture'] == architecture and data['corruption'] == common_corruption and data['severity'] == severity:
            return data['run_id']
            
    raise ValueError(f"Run not found for dataset={dataset}, architecture={architecture}, common_corruption={common_corruption}, severity={severity}")

In [274]:
def add_corruption(architecture, dataset):
    
    mlflow_data['architecture'].append(architecture)
    mlflow_data['backbone'].append("-")
    mlflow_data['type_backbone'].append("-")
    mlflow_data['dataset'].append(dataset)

    
    for common_corruption in common_corruptions:
        
        for severity in severities:

            # filter metadata to get the run_id
            run_id = get_run_id_from_metadata(metadata, architecture, dataset, common_corruption, severity)
                
            for metric, metric_architectures in metrics.items():
                
                # Schritt : Die spezifische Metrik abrufen
                metric_history = client.get_metric_history(run_id, metric)
                
                # Die Werte der Metrik abrufen
                values = [m.value for m in metric_history]
                metric_name = f"{metric}_{common_corruption}_sev{severity}"
                value = None
                
                if architecture not in metric_architectures:
                    value = "-"
                elif len(values) == 0:
                    raise ValueError(f"No values found for {metric}_{common_corruption}_sev{severity}, dataset: {dataset}, architecture: {architecture}, corruption: {common_corruption}, severity: {severity}")
                elif len(values) == 1:
                    value = values[0]
                else:
                    value = np.mean(values)

                print(f"\t{metric_name}: {value}")
            
                
                mlflow_data[metric_name].append(value)

        # clean/no_corruption
        # for metric in common_metrics:

In [275]:
for dataset in datasets:
    for architecture in architectures:
        print(architecture)
        add_corruption(
            architecture=architecture, 
            dataset=dataset)

cfnet
	test_EPE_0_gaussian_noise_sev1: 29.128144017257785
	test_Thres1_0_gaussian_noise_sev1: 0.6289255706887497
	test_Thres2_0_gaussian_noise_sev1: 0.5192185411514495
	test_Thres3_0_gaussian_noise_sev1: 0.46389480859115906
	test_D1_0_gaussian_noise_sev1: 0.4506077169438353
	test_EPE_0_shot_noise_sev1: 21.224237462912328
	test_Thres1_0_shot_noise_sev1: 0.56884721624633
	test_Thres2_0_shot_noise_sev1: 0.451610604729887
	test_Thres3_0_shot_noise_sev1: 0.39555092416106974
	test_D1_0_shot_noise_sev1: 0.38264496399758885
	test_EPE_0_impulse_noise_sev1: 9.353375955959207
	test_Thres1_0_impulse_noise_sev1: 0.4932858342982539
	test_Thres2_0_impulse_noise_sev1: 0.3575013061375858
	test_Thres3_0_impulse_noise_sev1: 0.29473457021012184
	test_D1_0_impulse_noise_sev1: 0.2810292312120955
	test_EPE_0_defocus_blur_sev1: 1.8367455396554018
	test_Thres1_0_defocus_blur_sev1: 0.12793524348660792
	test_Thres2_0_defocus_blur_sev1: 0.08507410749639609
	test_Thres3_0_defocus_blur_sev1: 0.06847321056170513
	te

In [276]:
mlflow_data

{'architecture': ['cfnet', 'gwcnet-g', 'sttr', 'sttr-light'],
 'backbone': ['-', '-', '-', '-'],
 'type_backbone': ['-', '-', '-', '-'],
 'dataset': ['sceneflow', 'sceneflow', 'sceneflow', 'sceneflow'],
 'test_EPE_0_gaussian_noise_sev1': [29.128144017257785,
  8.934847301858513,
  '-',
  '-'],
 'test_EPE_0_shot_noise_sev1': [21.224237462912328,
  8.893984444572396,
  '-',
  '-'],
 'test_EPE_0_impulse_noise_sev1': [9.353375955959207,
  8.458270091104835,
  '-',
  '-'],
 'test_EPE_0_defocus_blur_sev1': [1.8367455396554018,
  1.1894446005673922,
  '-',
  '-'],
 'test_EPE_0_glass_blur_sev1': [1.636481026312852,
  1.2957410747317482,
  '-',
  '-'],
 'test_EPE_0_motion_blur_sev1': [1.9148070458962114,
  1.8856493822087959,
  '-',
  '-'],
 'test_EPE_0_zoom_blur_sev1': [5.56685895554261, 4.606282368841105, '-', '-'],
 'test_EPE_0_snow_sev1': [3.0864337792658425, 2.408184996458704, '-', '-'],
 'test_EPE_0_frost_sev1': [5.528202583997146, 5.093937967830174, '-', '-'],
 'test_EPE_0_fog_sev1': [1.

In [ ]:
# Store mlflow_data as JSON
with open('mlflow_data.json', 'w') as f:
    json.dump(mlflow_data, f)